<a href="https://colab.research.google.com/github/win4r/mytest/blob/main/F5_TTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyngrok transformers gradio

In [ ]:
# 导入所需库
import os
from pyngrok import ngrok


In [ ]:
!apt-get update && apt-get install -y ffmpeg
!pip uninstall -y torch torchvision torchaudio transformers
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 transformers

# 克隆并安装项目
!git clone https://github.com/SWivid/F5-TTS.git
%cd F5-TTS
!pip install -e .


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 261 kB in 2s (138 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not s

In [ ]:
!ngrok config add-authtoken 2hKI7tLqJVdnbgM8pxM4nyYP7kQ_3vL3RWtqXQUUdwY5JE4nj

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# 配置 ngrok 和 gradio
import gradio as gr
from pyngrok import ngrok
import threading
import time
import socket
import requests
import subprocess

def is_port_in_use(port):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        return s.connect_ex(('localhost', port)) == 0

def wait_for_server(port, timeout=60):
    start_time = time.time()
    while time.time() - start_time < timeout:
        if is_port_in_use(port):
            try:
                response = requests.get(f'http://localhost:{port}')
                if response.status_code == 200:
                    return True
            except:
                pass
        time.sleep(2)
    return False

# 确保 ngrok 没有在运行
ngrok.kill()

# 在新线程中启动 Gradio
def run_gradio():
    # 使用 subprocess 而不是直接导入模块
    subprocess.run(['f5-tts_infer-gradio', '--port', '7860', '--host', '0.0.0.0'])

thread = threading.Thread(target=run_gradio)
thread.daemon = True
thread.start()

# 等待 Gradio 服务启动
print("等待 Gradio 服务启动...")
if wait_for_server(7860):
    print("Gradio 服务已启动")
    # 启动 ngrok
    public_url = ngrok.connect(7860)
    print(f"\n=== 访问信息 ===")
    print(f"Ngrok URL: {public_url}")
    print("===============\n")
else:
    print("Gradio 服务启动超时")

# 保持程序运行
while True:
    try:
        time.sleep(1)
    except KeyboardInterrupt:
        ngrok.kill()
        break


等待 Gradio 服务启动...
Gradio 服务已启动

=== 访问信息 ===
Ngrok URL: NgrokTunnel: "https://5534-34-19-6-34.ngrok-free.app" -> "http://localhost:7860"



In [ ]:
!f5-tts_infer-cli \
--model "F5-TTS" \
--ref_audio "/content/test.MP3" \
--ref_text "欢迎来到AI超元域,微软发布了一款基于大模型的屏幕解析工具OmniParser.这款工具是专为增强用户界面自动化而设计的它." \
--gen_text "欢迎来到AI超元域，今天将为大家详细演示另一款开源语音克隆项目。"

2024-10-31 09:17:22.203662: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-31 09:17:22.224170: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-31 09:17:22.230585: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-31 09:17:22.246129: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-31 09:17:23.396509: W tensorflow/compiler/tf2

In [ ]:
!pwd

/content/F5-TTS


In [ ]:
!ls

ckpts  data  Dockerfile  LICENSE  pyproject.toml  README.md  ruff.toml	src  tests
